In [ ]:
import sqlalchemy
from sqlalchemy import create_engine

In [ ]:
engine = create_engine('sqlite:///:memory:', echo = True)

In [ ]:
from sqlalchemy import MetaData, Table, Column, Integer, String, ForeignKey

In [ ]:
meta = MetaData()
users = Table('users', meta,
              Column('id', Integer, primary_key=True),
              Column('name', String),
              Column('fullname', String))

addr = Table('address', meta,
             Column('id', Integer, primary_key=True),
             Column('users.id', Integer, ForeignKey('users.id')),
            Column('email', String, nullable=True))

In [ ]:
meta.create_all(engine)

In [ ]:
meta.tables

In [ ]:
meta.bind # binding이 안되서 db와 연동 x

In [ ]:
print(users.insert())

In [ ]:
insert = users.insert().values(name='test', fullname = 'LJH')
print(insert)
print(insert.compile())
print(insert.compile().params)

In [ ]:
conn = engine.connect()

In [ ]:
conn.execute(insert)

In [ ]:
print(users.select().compile())

In [ ]:
cur = conn.execute(users.select())
print(cur)

In [ ]:
list(cur)

In [ ]:
cur = conn.execute(users.insert(),
                   {'name':'test2', 'fullname':'test2test2'})

In [ ]:
cur.lastrowid

In [ ]:
from sqlalchemy import select
print(select([users.c.name, users.c.fullname]))
print('---------')
cur = select([users.c.name, users.c.fullname]) # .c = column
print('---------')
print(list(cur))

In [ ]:
print(users.c.id == 1)
print((users.c.id == 1).compile().params)

In [ ]:
cur = conn.execute(users.select().where(users.c.id==1))
list(cur)

In [ ]:
conn.execute(addr.insert(), {'userid':1, 'email':'1@1.com'})
cur.lastrowid

In [ ]:
from sqlalchemy import join

In [ ]:
conn.execute(insert)

In [ ]:
print(users.join(addr))
print(select([users.c.name, users.c.fullname, addr.c.email])\.select_from(users.join(addr)))

In [ ]:
conn.execute(addr.insert(), {'usersid':1, 'email':'dsfasf'})
conn.execute(addr.select()).fetchall()

In [ ]:
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, ForeignKey, select, insert
engine = create_engine('sqlite:///ex3.db', echo=True)
engine

In [ ]:
meta = MetaData()

users = Table('user', meta,
              Column('id', Integer, Primary_key=True),
              Column('id', Integer, nullable = False)
             )

addr = Table('address', meta,
            Column('id', Integer, primary_key=True),
            Column('user_id', Integer, ForeignKey('user.id')),
            Column('email', String))

In [ ]:
meta.create_all(engine) # 진행과정 보기

In [ ]:
engine.execute(users.insert(), [{'name':'1'}, {'name': 2}])

In [ ]:
engine.execute(addr.insert(), 
              [{'user_id' : 1, 'email':'1@com'},
               {'user_id' : 1, 'email':'1@com'}])

In [ ]:
cur = engine.execute(select([users.c.name, addr.c.email]).select_from(users.join(addr)))
cur.fetchall()

In [ ]:
# session 오류나면 kernel reset 시키기

from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base

In [ ]:
engine = create_engine('sqlite:///ex4.db', echo=True)

In [ ]:
base = declarative_base()

In [ ]:
class User(base):
    __tablename__ = 'User'
    
    uid = Column('uid', Integer, primary_key=True)
    name = Column('name', String, nullable=False)
    password = Column('pass', String, nullable=False)
    
    def __repr__(self): # representation
        return 'UID={0}, NAME={1}, PASS={2}'.format(self.uid, self.name, self.password)

In [ ]:
User.__table__

In [ ]:
# in-memory table객체들 -> 실제 database table로 생성
base.metadata.create_all(bind=engine)

In [ ]:
## Metadata class에 등록된 table 객체 초기화할 떼
base.metadata.clear()

In [ ]:
user1 = User(name='jun', password='heui')

In [ ]:
user1

In [ ]:
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine)
session = Session()

In [ ]:
session.add(user1)

In [ ]:
session.commit() # 실제 table column 붙이고 데이터 만드는 과정 포함

In [ ]:
# Instances are considered dirty when they were modified but not deleted
# Return True if the given instance has locally modified attributes
session.dirty, session.is_modified 

In [ ]:
user1.name = '이준희'; session.commit() # update

In [ ]:
user1

In [ ]:
session.add_all([User(name='test2', password='123'),
               User(name='test3', password='123')])


In [ ]:
cur = session.query(User)
list(cur)

In [ ]:
#  Apply the given filtering criterion to a copy of this Query, using SQL expressions
[_ for _ in session.query(User).filter(User.name=='test2')]

In [ ]:
[_ for _ in session.query(User).filter(User.name.like('test%'))]

In [ ]:
# session 오류나면 kernel reset 시키기

from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import ForeignKey


engine = create_engine('sqlite:///ex5.db', echo=True)
base = declarative_base()

In [ ]:
## Metadata class에 등록된 table 객체 초기화할 떼
base.metadata.clear()

In [ ]:
class Artist(base):
    __tablename__ = 'ARTIST'
    
    pk = Column('ID', Integer, primary_key=True)
    name = Column('NAME', String)
    
    def __repr__(self):
        return 'PK:{0}, NAME:{1}'.format(self.pk, self.name)
    
class Album(base):
    __tablename__ = 'ALBUM'
    
    pk = Column('ID', Integer, primary_key=True)
    name = Column('NAME', String)
    fk = Column('FK', Integer, ForeignKey('ARTIST.ID'))
    
    def __repr__(self):
        return 'PK:{0}, NAME:{1}'.format(self.pk, self.name)

class Genre(base):
    __tablename__ = 'GENRE'
    
    pk = Column('ID', Integer, primary_key=True)
    name = Column('NAME', String)
    fk = Column('FK', Integer, ForeignKey('ARTIST.ID'))
    
    def __repr__(self):
        return 'PK:{0}, NAME:{1}, FK:{2}'.format(
            self.pk, self.name, self.fk)
    
class Track(base):
    __tablename__ = 'Track'
    
    pk = Column('ID', Integer, primary_key=True)
    name = Column('NAME', String)
    fk1 = Column('FK1', Integer, ForeignKey('ALBUM.ID'))
    fk2 = Column('FK2', Integer, ForeignKey('GENRE.ID'))
    
    def __repr__(self):
        return 'PK:{0}, NAME:{1}, FK1:{2}, FK2:{3}'.format(
            self.pk, self.name, self.fk1, self.fk2)

In [ ]:
base.metadata.create_all(bind=engine)

In [ ]:
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine)
session = Session()

In [ ]:
session.commit()

In [ ]:
# insert
a1 = Artist(name = 'a1')
a2 = Artist(name = 'a2')
session.add_all([a1, a2])
session.commit()

In [ ]:
g1 = Genre(name = 'g1')
g2 = Genre(name = 'g2')
session.add_all([g1, g2])
session.commit()

In [ ]:
session.add_all([
    Album(name='album1', fk=a1.pk),
    Album(name='album2', fk=a2.pk),
])
session.commit()

In [ ]:
[_[0] for _ in session.query(Album)]

In [ ]:
al1 = session.query(Album).filter(Album.fk = a1.pk).one()
al2 = session.query(Album).filter(Album.fk = a2.pk).one()

In [ ]:
session.add_all([
    Track(name='track1', fk=al1.pk, fk2=g1.pk),
    Track(name='track2', fk=al1.pk, fk2=g1.pk),
    Track(name='track3', fk=al1.pk, fk2=g1.pk),
    Track(name='track4', fk=al1.pk, fk2=g1.pk),
])
session.commit()

In [ ]:
tracklist = session.query(Track).all()
tracklist[2].fk1 = tracklist[3].fk1 = 4
session.commit()

In [ ]:
a1, a2, 

In [ ]:
list(session.query(Track))

In [ ]:
session.query(Artist.name, Album.name, Genre.name, Track.name)\
.select_from(Track)\
.join(Genre).join(Album).join(Artist).all()

for _ in result:
    print('Artist:{0}, Album:{1}, Genre:{2}, Track:{3}'.format(
    _[0],_[1],_[2],_[3]))

In [ ]:
## 나중에 다시 받기

In [1]:
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import ForeignKey
from sqlalchemy.orm import relationship, backref

In [2]:
engine = create_engine('sqlite:///playlist.db', echo=True)
base = declarative_base()
## Metadata class에 등록된 table 객체 초기화할 떼
base.metadata.clear()

In [3]:
class Artist(base):
    __tablename__ = 'Artist'
    
    pk = Column('id', Integer, primary_key=True)
    name = Column('title', String)
    albumlist = relationship('Album', back_populates='artist')  
    
    def __repr__(self):
        return 'PK:{0}, NAME:{1}'.format(self.pk, self.name)

class Genre(base):
    __tablename__ = 'Genre'
    
    pk = Column('id', Integer, primary_key=True)
    name = Column('title', String)
    tracklist = relationship('Track', back_populates='genre')
    
    def __repr__(self):
        return 'PK:{0}, NAME:{1}'.format(self.pk, self.name)

class Album(base):
    __tablename__ = 'Album'
    
    pk = Column('id', Integer, primary_key=True)
    name = Column('title', String)
    fk = Column('Artist_id', Integer, ForeignKey('Artist.id'))
    artist = relationship('Artist', back_populates='albumlist',
                         uselist=False) # 1: n
    tracklist = relationship('Track', back_populates='album')
    
    def __repr__(self):
        return 'PK:{0}, NAME:{1}, FK:{2}'.format(
            self.pk, self.name, self.fk)


    
class Track(base):
    __tablename__ = 'Track'
    
    pk = Column('id', Integer, primary_key=True)
    name = Column('title', String)
    fk1 = Column('Album_id', Integer, ForeignKey('Album.id'))
    fk2 = Column('Genre_id', Integer, ForeignKey('Genre.id'))
    album = relationship('Album', back_populates='tracklist',
                            uselist=False)
    genre = relationship('Genre', back_populates='tracklist',
                            uselist=False)
    
    def __repr__(self):
        return 'PK:{0}, NAME:{1}, FK1:{2}, FK2:{3}'.format(
            self.pk, self.name, self.fk1, self.fk2)
    
    

In [4]:
base.metadata.create_all(bind=engine)

2020-07-15 12:20:28,606 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-07-15 12:20:28,609 INFO sqlalchemy.engine.base.Engine ()
2020-07-15 12:20:28,609 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-07-15 12:20:28,610 INFO sqlalchemy.engine.base.Engine ()
2020-07-15 12:20:28,611 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Artist")
2020-07-15 12:20:28,611 INFO sqlalchemy.engine.base.Engine ()
2020-07-15 12:20:28,613 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Genre")
2020-07-15 12:20:28,614 INFO sqlalchemy.engine.base.Engine ()
2020-07-15 12:20:28,615 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Album")
2020-07-15 12:20:28,615 INFO sqlalchemy.engine.base.Engine ()
2020-07-15 12:20:28,616 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Track")
2020-07-15 12:20:28,617 INFO sqlalchemy.engine.base.Engine ()


In [5]:
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine)
session = Session()


In [ ]:
a1 = Artist(name='A1')
session.add(a1)

In [6]:
genrelist = session.query(Genre).all()

2020-07-15 12:20:29,816 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-07-15 12:20:29,817 INFO sqlalchemy.engine.base.Engine SELECT "Genre".id AS "Genre_id", "Genre".title AS "Genre_title" 
FROM "Genre"
2020-07-15 12:20:29,817 INFO sqlalchemy.engine.base.Engine ()


In [7]:
genrelist[1].tracklist[0], genrelist[1].tracklist[0].album.name

2020-07-15 12:20:34,409 INFO sqlalchemy.engine.base.Engine SELECT "Track".id AS "Track_id", "Track".title AS "Track_title", "Track"."Album_id" AS "Track_Album_id", "Track"."Genre_id" AS "Track_Genre_id" 
FROM "Track" 
WHERE ? = "Track"."Genre_id"
2020-07-15 12:20:34,410 INFO sqlalchemy.engine.base.Engine (2,)
2020-07-15 12:20:34,412 INFO sqlalchemy.engine.base.Engine SELECT "Album".id AS "Album_id", "Album".title AS "Album_title", "Album"."Artist_id" AS "Album_Artist_id" 
FROM "Album" 
WHERE "Album".id = ?
2020-07-15 12:20:34,412 INFO sqlalchemy.engine.base.Engine (1,)


(PK:3, NAME:About to Rock, FK1:1, FK2:2, 'IV')

In [9]:
# session.add(Track(name='TEST',
#                  fk1=genrelist[1].tracklist[0].album.pk,
#                  fk2=genrelist[1].pk))
# session.commit()